<a href="https://colab.research.google.com/github/Kim-Dong-Jun99/DSTermProject/blob/main/term_project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# commit and merge test

In [ ]:
#공유 드라이브 연동

from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive', force_remount = True)  # 공유 드라이브

df_2015_path = r'/content/gdrive/Shareddrives/data_science/dataset/2015.csv'
df_2016_path = r'/content/gdrive/Shareddrives/data_science/dataset/2016.csv'
df_2017_path = r'/content/gdrive/Shareddrives/data_science/dataset/2017.csv'


Mounted at /content/gdrive


In [ ]:
df_2015 = pd.read_csv(df_2015_path)
df_2016 = pd.read_csv(df_2016_path)
df_2017 = pd.read_csv(df_2017_path)

# TermProject

## Source and explanation of the dataset

The dataset is 'World Happiness Report' from Kaggle. 'World Happiness Report' has data of world happiness from 2015 to 2019

https://www.kaggle.com/datasets/unsdsn/world-happiness?datasetId=894&sortBy=voteCount&searchQuery=predict

The columns of each dataset include 'Country', 'Region', 'Happiness Rank', 'Happiness Score', 'Economy', 'Family', 'Health', 'Freedom', 'Trust', 'Generosity', 'Dystopia Residual' etc.

It is arranged in descending order according to the happiness score. The happiness score is the sum of the scores of other numerical features (except for Hapiness Rank).

The World Happiness Report is a landmark survey of the state of global happiness. The reports review the state of happiness in the world today and show how the


# Objective of our analysis
Analyze dataset to figure out characteristics of each continent in dataset then predict continent value of countries in dataset of 2017 using 2015, 2016 dataset for training

We will use clustering and classification algorithm to predict continent of countries in dataset of 2017 using 2015, 2016 happiness data report

Also, we will automate procedure of choosing best combination of scaler and algorithm that gives best result

# 데이터 통일하는 과정
# process of unification

## 각 데이터세트마다 존재하는 나라가 달라서 데이터세트에 있는 나라들을 통일하기 위한 코드
## countries in each datasets are differenet, must be unificated before modeling 

In [ ]:
# 각 데이터프레임들의 나라를 저장하는 리스트
# list for saving countries in each datasets
country_15 = []
country_16 = []
country_17 = []

for i in range(len(df_2015)):
    country_15.append(df_2015.iloc[i][0])

for i in range(len(df_2016)):
    country_16.append(df_2016.iloc[i][0])

for i in range(len(df_2017)):
    country_17.append(df_2017.iloc[i][0])

# 공통된 나라를 저장하기 위한 리스트
# list to save common countries in dataset
common = []
for i in country_15:
    if (i  in country_17 and i in country_16):
        common.append(i)


common.sort()

# 공통된 나라만 데이터세트에 남기기 위해서 공통으로 존재하지 않는 나라를 저장하기 위한 리스트
# list for saving countries that are non-common
drop15 = []
drop16 = []
drop17 = []

for i in range(len(df_2015)):
    if(df_2015.iloc[i][0] not in common):
        drop15.append(i)

for i in range(len(df_2016)):
    if(df_2016.iloc[i][0] not in common):
        drop16.append(i)

for i in range(len(df_2017)):
    if(df_2017.iloc[i][0] not in common):
        drop17.append(i)

print(len(df_2015))
print(len(df_2016))
print(len(df_2017))




# 공통된 나라가 common 리스트에 저장되었고, common 리스트에 속하지 않는 나라들은 drop
# code for dropping countrires that are not in common list

In [ ]:
# drop15,drop16, drop17에 드랍해야할 나라들이 들어가있는데 그것을 적용해서 드랍할 나라들 드랍하기
# drop countries in each dataframe
df_2015.drop(drop15, axis = 0,inplace=True)
df_2016.drop(drop16, axis = 0,inplace=True)
df_2017.drop(drop17, axis = 0,inplace=True)

print(len(df_2015))
print(len(df_2016))
print(len(df_2017))


# 158, 157, 155개였던 나라들이 146개로 통일된 것을 볼 수 있다
# you can see that number of countries in each dataset is identical

# 이제 서로 다른 컬럼 명을 통일하기 인덱스 정리하기
# now we should unificate name of columns and reset index

In [ ]:
# 나라들이 드랍됐으니, index를 정리
# resetting index because countries has been dropped

df_2015.reset_index(drop=True,inplace = True)
df_2016.reset_index(drop=True, inplace = True)
df_2017.reset_index(drop=True, inplace = True)



In [ ]:
# 2017년도 데이터의 컬럼 명들이 다르기에 컬럼 명을 통일해주는 코드
# column name in 2017 is different, code for changing 2017 data column name
df_2017.rename(columns = {"Happiness.Rank":"Happiness Rank", "Happiness.Score":"Happiness Score","Economy..GDP.per.Capita.":"Economy (GDP per Capita)",
                          "Health..Life.Expectancy.":"Health (Life Expectancy)","Trust..Government.Corruption.":"Trust (Government Corruption)","Dystopia.Residual":"Dystopia Residual"}, inplace=True)
# 표준편차 같은 의미 없는 컬럼 들은 드랍하기
# dropping meaningless columns
c_drop15 = ['Standard Error']
c_drop16 = ['Lower Confidence Interval','Upper Confidence Interval']
c_drop17 = ['Whisker.high', 'Whisker.low']

df_2015.drop(c_drop15, axis=1,inplace=True)
df_2016.drop(c_drop16, axis=1,inplace=True)
df_2017.drop(c_drop17, axis=1,inplace=True)

# 칼럼까지 통일했으니, 2017년도에 없는 지역 컬럼을 추가해줘야하고, region 컬럼에 있는 값을 대륙으로 변경해야한다

# column name is unificated, now add region column in 2017 data and value in region column must be continent

In [ ]:
# 2017년도 데이터에 지역 컬럼을 추가해주기 위한 딕셔너리
# dictonary for adding region data in 2017 dataframe
region_dic = {}
# 각 년도별 데이터프레임에 있는 나라들이 동일하므로, 2017년도 데이터에 지역 컬럼을 추가해주는 코드
# adding region column to dataset of 2017 dataframe
for i in range(len(df_2016)):
  country = df_2016.iloc[i][0]
  region = df_2016.iloc[i][1]
  region_dic[country] = region

for i in range(len(df_2017)):
  df_2017.loc[i,'Region'] = region_dic[df_2017.loc[i,'Country']]

df_2017.head()



# 2017년도 데이터에 지역 컬럼이 추가된 것을 확인할 수 있다, 이제 지역 컬럼에 있는 값을 지역이 아니라 대륙으로 변경해주자

# you can see that region column has been added to 2017 dataframe, now values in region column must be continent, 

In [ ]:
my_list=list()
# 지역 값을 대륙으로 변경시켜주는 코드
# changing region value into continent value
for index in df_2015["Region"]:
    if "Africa" in index:
        my_list.append("Africa")
    elif "Europe" in index:
        my_list.append("Europe")
    elif "Antartica"in index:
        my_list.append("Antartica")
    elif "Australia"in index:
        my_list.append("Australia")
    elif "North America"in index:
        my_list.append("North America")
    elif "Latin America" in index:
        my_list.append("South America")
    elif "Asia" in index:
        my_list.append("Asia")
df_2015["Region"]=my_list

my_list=list()
for index in df_2016["Region"]:
    if "Africa" in index:
        my_list.append("Africa")
    elif "Europe" in index:
        my_list.append("Europe")
    elif "Antartica"in index:
        my_list.append("Antartica")
    elif "Australia"in index:
        my_list.append("Australia")
    elif "North America"in index:
        my_list.append("North America")
    elif "Latin America" in index:
        my_list.append("South America")
    elif "Asia" in index:
        my_list.append("Asia")
df_2016["Region"]=my_list

my_list=list()
for index in df_2017["Region"]:
    if "Africa" in index:
        my_list.append("Africa")
    elif "Europe" in index:
        my_list.append("Europe")
    elif "Antartica"in index:
        my_list.append("Antartica")
    elif "Australia"in index:
        my_list.append("Australia")
    elif "North America"in index:
        my_list.append("North America")
    elif "Latin America" in index:
        my_list.append("South America")
    elif "Asia" in index:
        my_list.append("Asia")
df_2017["Region"]=my_list


In [ ]:
# 이제 데이터 세트를 보기 편하게 해주기 위해서 컬럼 순서를 2015년 데이터프레임에 맞춰주는 코드
# resetting order of dataframe column to 2015 data
df_2016 = df_2016[['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual']]
df_2017 = df_2017[['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual']]

In [ ]:
df_2015.head()

In [ ]:
df_2016.head()

In [ ]:
df_2017.head()

# 데이터 프레임 통일 끝!
# dataframe unifiacation done

In [ ]:
# 처리한 데이터 프레임을 재활용하기 위해서 피클 파일로 저장하는 코드
# code for saving data into pickle file

# df_2015.to_pickle('/content/gdrive/Shareddrives/data_science/dataset/cleaned_2015.pkl')
# df_2016.to_pickle('/content/gdrive/Shareddrives/data_science/dataset/cleaned_2016.pkl')
# df_2017.to_pickle('/content/gdrive/Shareddrives/data_science/dataset/cleaned_2017.pkl')

In [ ]:
cleaned_2015 = pd.read_pickle('/content/gdrive/Shareddrives/data_science/dataset/cleaned_2015.pkl')
cleaned_2016 = pd.read_pickle('/content/gdrive/Shareddrives/data_science/dataset/cleaned_2016.pkl')
cleaned_2017 = pd.read_pickle('/content/gdrive/Shareddrives/data_science/dataset/cleaned_2017.pkl')